In [1]:
import pandas as pd
pd.set_option("display.max_columns", None)
import re

In [2]:
#Rive Droite = 884
#Chateau = 885
#Chantiers = 886
#Porchefontaine = 679
#Montreuil = 566
stif_code = {
    "Versailles Rive Droite" : "884",
    "Versailles Château Rive Gauche" : "885",
    "Versailles Chantiers" : "886",
    "Porchefontaine" : "679",
    "Montreuil" : "566"
}

stif_code_reverse = {value:key for key, value in stif_code.items()}

In [3]:
test = pd.read_csv('ef_versailles.csv')

In [4]:
test.head()

,Station_name,Network,Longitude,Latitude,City,Traffic_2015,Traffic_2016,Traffic_2017,Traffic_2018,Traffic_2019,Traffic_2020,Traffic_2021,Postcode,Traffic_2022,dept,Traffic_pre_cov,Traffic_pre_cov_sd,covid_drop_2020,covid_drop_2021,Traffic_2020_2021,Origin_id,Rank_rer_2015,Rank_rer_2016,Rank_rer_2017,Rank_rer_2018,Rank_rer_2019,Rank_rer_2020,Rank_rer_2021,Rank_rer_2022
0,Versailles Chantiers,RER,2.135160,48.795690,versailles,24191310.0,24132107.0,23974181.0,23336771.0,22909416.0,12376593,15922909,78000.0,18934500.0,78.0,23708757.0,501966.560353,-0.477974,-0.328395,0.286534,531,8.0,9.0,9.0,10.0,10.0,8.0,11.0,8.0
1,Versailles Château Rive Gauche,RER,2.128880,48.800330,versailles,8748796.0,8367046.0,8009738.0,7520468.0,7015859.0,2886249,3310046,78000.0,5795124.0,78.0,7932381.4,611741.140672,-0.636143,-0.582717,0.146833,457,41.0,49.0,55.0,59.0,64.0,71.0,103.0,62.0
2,Versailles Rive Droite,RER,2.135087,48.809482,versailles,5357929.0,5327810.0,5562438.0,5705373.0,5855418.0,2039014,4698156,78000.0,5530365.0,78.0,5561793.6,201563.810161,-0.633389,-0.155280,1.304131,458,83.0,85.0,81.0,81.0,79.0,109.0,71.0,64.0
3,Montreuil (transilien),RER,2.152993,48.805941,versailles,2248554.0,2137642.0,2192249.0,2204405.0,2226698.0,748793,1897965,78000.0,2010237.0,78.0,2201909.6,37470.103948,-0.659935,-0.138037,1.534699,423,183.0,198.0,196.0,196.0,194.0,231.0,178.0,156.0
4,Porchefontaine,RER,2.152379,48.796528,versailles,956155.0,946641.0,933001.0,902371.0,875987.0,398498,545751,78000.0,723565.0,78.0,922831.0,29643.906328,-0.568179,-0.408612,0.369520,663,264.0,267.0,273.0,275.0,279.0,276.0,301.0,241.0


In [5]:
traffic = pd.read_csv('ef_versailles.csv')
traffic = traffic.drop(["Network", "Longitude", 'Latitude', "City", "Postcode", "dept"], axis = 1)
traffic = traffic.rename(columns = {excol:str(excol).lower() for excol in traffic.columns})
traffic[["traffic_2015","traffic_2016","traffic_2017","traffic_2018","traffic_2019","traffic_2020","traffic_2021","traffic_2022","traffic_pre_cov","traffic_pre_cov_sd"]] = traffic[["traffic_2015","traffic_2016","traffic_2017","traffic_2018","traffic_2019","traffic_2020","traffic_2021","traffic_2022","traffic_pre_cov","traffic_pre_cov_sd"]].astype('int')
traffic[["rank_rer_2015","rank_rer_2016","rank_rer_2017","rank_rer_2018","rank_rer_2019","rank_rer_2020","rank_rer_2021","rank_rer_2022"]] = traffic[["rank_rer_2015","rank_rer_2016","rank_rer_2017","rank_rer_2018","rank_rer_2019","rank_rer_2020","rank_rer_2021","rank_rer_2022"]].astype('int')
traffic.loc[traffic["origin_id"] == 531, "station_name"] = "Versailles Chantiers"
traffic.loc[traffic["origin_id"] == 457, "station_name"] = "Versailles Château Rive Gauche"
traffic.loc[traffic["origin_id"] == 458, "station_name"] = "Versailles Rive Droite"
traffic.loc[traffic["origin_id"] == 423, "station_name"] = "Montreuil"
traffic.loc[traffic["origin_id"] == 663, "station_name"] = "Porchefontaine"
traffic['station_id'] = traffic['station_name'].map(stif_code)
traffic = traffic[["station_name", "station_id", "origin_id", "traffic_2015","traffic_2016","traffic_2017","traffic_2018","traffic_2019","traffic_2020","traffic_2021","traffic_2022","traffic_pre_cov","traffic_pre_cov_sd", "covid_drop_2020","covid_drop_2021","traffic_2020_2021", "rank_rer_2015","rank_rer_2016","rank_rer_2017","rank_rer_2018","rank_rer_2019","rank_rer_2020","rank_rer_2021","rank_rer_2022"]]
traffic = traffic.drop(["traffic_2015","traffic_2016","traffic_2017","traffic_2018","traffic_2019","traffic_2020","traffic_2021","traffic_2022","traffic_pre_cov","traffic_pre_cov_sd", "covid_drop_2020","covid_drop_2021","traffic_2020_2021"], axis = 1)
traffic = traffic.rename(columns = {"rank_rer_2015" : 2015, "rank_rer_2016" : 2016, "rank_rer_2017" : 2017, "rank_rer_2018" : 2018, "rank_rer_2019" : 2019, "rank_rer_2020" : 2020, "rank_rer_2021" : 2021, "rank_rer_2022" : 2022})
traffic = pd.melt(traffic, id_vars = ['station_name', 'station_id', 'origin_id'], var_name = "year", value_name = "rank")
traffic.head()

,station_name,station_id,origin_id,year,rank
0,Versailles Chantiers,886,531,2015,8
1,Versailles Château Rive Gauche,885,457,2015,41
2,Versailles Rive Droite,884,458,2015,83
3,Montreuil,566,423,2015,183
4,Porchefontaine,679,663,2015,264


In [6]:
network_dict = {
    '110' : "metro", 
    '803' : "RER C", 
    '854' : "L & J", 
    '804' : "RER D", 
    '801' : "RER A", 
    '802' : "RER B", 
    '853' : "H", 
    '852' : "N", 
    '850' : "P", 
    '805' : "RER E",
    '851' : "R", 
    '822' : "T4"
}

<h4>csv import - all stations</h4>

In [7]:
## EVERY STATION 

# 2015

df15_s1 = pd.read_csv('original_datasets/data-rf-2015/2015S1_NB_FER.csv', sep = ";")
df15_s2 = pd.read_csv('original_datasets/data-rf-2015/2015S2_NB_FER.csv', sep = ";")

# 2016

df16_s1 = pd.read_csv('original_datasets/data-rf-2016/2016S1_NB_FER.txt', sep = "\t")
df16_s2 = pd.read_csv('original_datasets/data-rf-2016/2016S2_NB_FER.txt', sep = "\t")

# 2017

df17_s1 = pd.read_csv('original_datasets/data-rf-2017/2017S1_NB_FER.txt', sep = "\t")
df17_s2 = pd.read_csv('original_datasets/data-rf-2017/2017_S2_NB_FER.txt', sep = "\t")

# 2018

df18_s1 = pd.read_csv('original_datasets/data-rf-2018/2018_S1_NB_FER.txt', sep = "\t")
df18_s2 = pd.read_csv('original_datasets/data-rf-2018/2018_S2_NB_FER.txt', sep = "\t")

# 2019

df19_s1 = pd.read_csv('original_datasets/data-rf-2019/2019_S1_NB_FER.txt', sep = "\t")
df19_s2 = pd.read_csv('original_datasets/data-rf-2019/2019_S2_NB_FER.txt', sep = "\t")

# 2020

df20_s1 = pd.read_csv('original_datasets/data-rf-2020/2020_S1_NB_FER.txt', sep = "\t")
df20_s2 = pd.read_csv('original_datasets/data-rf-2020/2020_S2_NB_FER.txt', sep = "\t")

# 2021

df21_s1 = pd.read_csv('original_datasets/data-rf-2021/2021_S1_NB_FER.txt', sep = "\t")
df21_s2 = pd.read_csv('original_datasets/data-rf-2021/2021_S2_NB_FER.txt', sep = "\t")

# 2022

df22_s1 = pd.read_csv('original_datasets/data-rf-2022/2022_S1_NB_FER.txt', sep = "\t")
df22_s2 = pd.read_csv('original_datasets/data-rf-2022/2022_S2_NB_FER.txt', sep = ";")

# 2023

df23_s1_all = pd.read_csv('original_datasets/data-rf-2023/2023_S1_NB_FER.csv', sep = ";")

df2_all = pd.concat([df15_s1, df15_s2, df16_s1, df16_s2, df17_s1, df17_s2, df18_s1, df18_s2,
                df19_s1, df19_s2, df20_s1, df20_s2, df21_s1, df21_s2, df22_s1, df22_s2,
               ]).reset_index(drop = True)

df2_all['JOUR'] = pd.to_datetime(df2_all['JOUR'], format = '%d/%m/%Y')
df23_s1_all['JOUR'] = pd.to_datetime(df23_s1_all['JOUR'], format = '%Y-%m-%d')


In [8]:
stations_L = ["PONT-CARDINET", "ASNIERES", "LA DEFENSE-GRANDE ARCHE", "SAINT-CLOUD", "SURESNES-MONT VALERIEN",
              "CLICHY-LEVALLOIS", "SEVRES-VILLE-D'AVRAY", "VIROFLAY RIVE DROITE", "CHAVILLE RIVE DROITE",
              "VERSAILLES RIVE DROITE", "SAINT-LAZARE", "SEVRES VILLE", "ST CLOUD", "SURESNES", "VERSAILLES RD",
              "VIROFLAY RD", "CHAVILLE RD", "CLICHY LEVALLOI", "ASNIERES SUR S", "DEFENSE"]

L = pd.concat([df2_all,df23_s1_all]).reset_index(drop = True)
L = L[L["LIBELLE_ARRET"].isin(stations_L)]
L['CODE_STIF_RES'] = L['CODE_STIF_RES'].str.strip()
L = L[~L["CODE_STIF_RES"].isin(["110", "ND"])]                # Deleting rows from metro stations

L = L.drop(["CODE_STIF_TRNS", "ID_REFA_LDA", "lda"], axis = 1)
L = L.rename(columns = {"JOUR" : "date", "CODE_STIF_RES" : "network",  "CODE_STIF_ARRET" : "station_id", "LIBELLE_ARRET" : "station_name", "CATEGORIE_TITRE" : "pass_type", "NB_VALD" : "n_passengers"})
L['network'] = L['network'].map(network_dict)   
len(L[L['date'] == "01/10/2019"]['station_name'].unique())

10

<h4>Dataset creation: idf</h4>

In [9]:
idf = pd.concat([df2_all,df23_s1_all]).reset_index(drop = True)

idf['CODE_STIF_RES'] = idf['CODE_STIF_RES'].str.strip()
idf = idf[~idf["CODE_STIF_RES"].isin(["110", "ND", "822"])]                # Deleting rows from metro stations
idf = idf.drop(["CODE_STIF_TRNS", "ID_REFA_LDA", "lda"], axis = 1)
idf = idf.rename(columns = {"JOUR" : "date", "CODE_STIF_RES" : "network",  "CODE_STIF_ARRET" : "station_id", "LIBELLE_ARRET" : "station_name", "CATEGORIE_TITRE" : "pass_type", "NB_VALD" : "n_passengers"})
idf['network'] = idf['network'].map(network_dict)
idf['n_passengers'] = idf.apply(lambda row: 2 if row['n_passengers'] == "Moins de 5" else row['n_passengers'], axis = 1)         #Arbitrarily setting "< 5" to 2 validations
idf['n_passengers'] = idf['n_passengers'].astype("int")

idf['year'] = idf['date'].dt.year
idf['day'] = idf['date'].dt.day
idf['month'] = idf['date'].dt.month
idf['day_name'] = idf['date'].dt.day_name()
idf['month_name'] = idf['date'].dt.month_name()

idf = idf.pivot_table(index=['date', 'station_id', 'station_name', 'year', 'day', 'month', 'day_name', 'month_name'],
                                  columns='pass_type', values='n_passengers', aggfunc='sum').reset_index()
idf.columns.name = None

idf[["?", "AMETHYSTE", "AUTRE TITRE", "FGT", "IMAGINE R", "NAVIGO", "NAVIGO JOUR", "NON DEFINI", "TST"]] = idf[["?", "AMETHYSTE", "AUTRE TITRE", "FGT", "IMAGINE R", "NAVIGO", "NAVIGO JOUR", "NON DEFINI", "TST"]].fillna(0).astype("int")

aggregation = {
    "station_id" : "first",
    "station_name" : "first",
    "?" : "sum",
    "AMETHYSTE" : "sum",
    "AUTRE TITRE" : "sum",
    "FGT" : "sum",
    "IMAGINE R" : "sum",
    "NAVIGO" : "sum",
    "NAVIGO JOUR" : "sum",
    "NON DEFINI": "sum",
    "TST" : "sum",
    'year': "first",
    'day' : "first",
    'month' : "first",
    'day_name' : "first",
    'month_name' : "first"
}

idf = idf.groupby('date').agg(aggregation).reset_index()
idf['station_name'] = "all_idf"
idf[['station_id', "network"]] = "N/A"

idf = idf.drop('network', axis = 1)

idf['social'] = idf['FGT'] + idf['TST']
idf['other'] = idf['?'] + idf['AUTRE TITRE'] + idf['NON DEFINI']
idf = idf.drop(['?', "AUTRE TITRE", "FGT", "NON DEFINI", "TST"], axis = 1)
idf = idf.rename(columns = {"AMETHYSTE" : "amethyste", "IMAGINE R" : "imagine_r", "NAVIGO": "navigo", "NAVIGO JOUR": "navigo_jour"})
idf["daily_traffic"] = idf["amethyste"] + idf["imagine_r"] + idf["navigo"] + idf["navigo_jour"] + idf["social"] + idf["other"]

S1 = ["January", "February", "March", "April", "May", "June"]
S2 = ["July", "August", "September", "October", "November", "December"]

idf["semester"] = idf.apply(lambda row: 1 if row['month_name'] in S1 else 2 if row['month_name'] in S2 else None, axis = 1)


idf

,date,station_id,station_name,amethyste,imagine_r,navigo,navigo_jour,year,day,month,day_name,month_name,social,other,daily_traffic,semester
0,2015-01-01,N/A,all_idf,10286,56583,141278,0,2015,1,1,Thursday,January,56407,9718,274272,1
1,2015-01-02,N/A,all_idf,23737,149382,618541,0,2015,2,1,Friday,January,156352,31513,979525,1
2,2015-01-03,N/A,all_idf,22099,133366,401524,0,2015,3,1,Saturday,January,140141,18054,715184,1
3,2015-01-04,N/A,all_idf,19311,90120,290605,0,2015,4,1,Sunday,January,114939,15951,530926,1
4,2015-01-05,N/A,all_idf,28160,316092,1274576,0,2015,5,1,Monday,January,209630,68042,1896500,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3098,2023-06-26,N/A,all_idf,34600,438986,1423915,7855,2023,26,6,Monday,June,293630,132851,2331837,1
3099,2023-06-27,N/A,all_idf,36390,464496,1518545,7270,2023,27,6,Tuesday,June,296672,138926,2462299,1
3100,2023-06-28,N/A,all_idf,35273,457786,1374888,7201,2023,28,6,Wednesday,June,261351,127937,2264436,1
3101,2023-06-29,N/A,all_idf,36323,473805,1475731,8048,2023,29,6,Thursday,June,292223,138310,2424440,1


<h4>Dataset creation: line L</h4>

In [10]:
stations_L = ["PONT-CARDINET", "ASNIERES", "LA DEFENSE-GRANDE ARCHE", "SAINT-CLOUD", "SURESNES-MONT VALERIEN",
              "CLICHY-LEVALLOIS", "SEVRES-VILLE-D'AVRAY", "VIROFLAY RIVE DROITE", "CHAVILLE RIVE DROITE",
              "VERSAILLES RIVE DROITE", "SAINT-LAZARE", "SEVRES VILLE", "ST CLOUD", "SURESNES", "VERSAILLES RD",
              "VIROFLAY RD", "CHAVILLE RD", "CLICHY LEVALLOI", "ASNIERES SUR S", "DEFENSE"]

L = pd.concat([df2_all,df23_s1_all]).reset_index(drop = True)
L = L[L["LIBELLE_ARRET"].isin(stations_L)]
L['CODE_STIF_RES'] = L['CODE_STIF_RES'].str.strip()
L = L[~L["CODE_STIF_RES"].isin(["110", "ND"])]                # Deleting rows from metro stations

L = L.drop(["CODE_STIF_TRNS", "ID_REFA_LDA", "lda"], axis = 1)
L = L.rename(columns = {"JOUR" : "date", "CODE_STIF_RES" : "network",  "CODE_STIF_ARRET" : "station_id", "LIBELLE_ARRET" : "station_name", "CATEGORIE_TITRE" : "pass_type", "NB_VALD" : "n_passengers"})
L['network'] = L['network'].map(network_dict)
L['n_passengers'] = L.apply(lambda row: 2 if row['n_passengers'] == "Moins de 5" else row['n_passengers'], axis = 1)         #Arbitrarily setting "< 5" to 2 validations
L['n_passengers'] = L['n_passengers'].astype("int")

L['year'] = L['date'].dt.year
L['day'] = L['date'].dt.day
L['month'] = L['date'].dt.month
L['day_name'] = L['date'].dt.day_name()
L['month_name'] = L['date'].dt.month_name()

L = L.pivot_table(index=['date', 'station_id', 'station_name', 'year', 'day', 'month', 'day_name', 'month_name'],
                                  columns='pass_type', values='n_passengers', aggfunc='sum').reset_index()
L.columns.name = None

L[["?", "AMETHYSTE", "AUTRE TITRE", "FGT", "IMAGINE R", "NAVIGO", "NAVIGO JOUR", "NON DEFINI", "TST"]] = L[["?", "AMETHYSTE", "AUTRE TITRE", "FGT", "IMAGINE R", "NAVIGO", "NAVIGO JOUR", "NON DEFINI", "TST"]].fillna(0).astype("int")

L = L.groupby('date').agg(aggregation).reset_index()
L['station_name'] = "line_L"
L[['station_id', "network"]] = "N/A"



L['social'] = L['FGT'] + L['TST']
L['other'] = L['?'] + L['AUTRE TITRE'] + L['NON DEFINI']
L = L.drop(['?', "AUTRE TITRE", "FGT", "NON DEFINI", "TST"], axis = 1)
L = L.rename(columns = {"AMETHYSTE" : "amethyste", "IMAGINE R" : "imagine_r", "NAVIGO": "navigo", "NAVIGO JOUR": "navigo_jour"})
L["daily_traffic"] = L["amethyste"] + L["imagine_r"] + L["navigo"] + L["navigo_jour"] + L["social"] + L["other"]
L = L.drop("network", axis = 1)
S1 = ["January", "February", "March", "April", "May", "June"]
S2 = ["July", "August", "September", "October", "November", "December"]

#L["semester"] = L.apply(lambda row: 1 if row['month_name'] in S1 else 2 if row['month_name'] in S2 else None, axis = 1)

L

,date,station_id,station_name,amethyste,imagine_r,navigo,navigo_jour,year,day,month,day_name,month_name,social,other,daily_traffic
0,2015-01-01,N/A,line_L,515,3961,10244,0,2015,1,1,Thursday,January,2960,385,18065
1,2015-01-02,N/A,line_L,1259,11259,58046,0,2015,2,1,Friday,January,8726,1502,80792
2,2015-01-03,N/A,line_L,1178,9574,31768,0,2015,3,1,Saturday,January,7832,721,51073
3,2015-01-04,N/A,line_L,900,6464,21858,0,2015,4,1,Sunday,January,5975,613,35810
4,2015-01-05,N/A,line_L,1622,20896,138957,0,2015,5,1,Monday,January,12054,3169,176698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3098,2023-06-26,N/A,line_L,1858,33120,132157,508,2023,26,6,Monday,June,16623,8590,192856
3099,2023-06-27,N/A,line_L,2069,35059,144402,560,2023,27,6,Tuesday,June,16324,8304,206718
3100,2023-06-28,N/A,line_L,2066,35289,131727,537,2023,28,6,Wednesday,June,14365,7976,191960
3101,2023-06-29,N/A,line_L,2084,39970,154924,721,2023,29,6,Thursday,June,18151,9449,225299


<h4>Dataset creation: line N</h4>

In [11]:
N = pd.concat([df2_all,df23_s1_all]).reset_index(drop = True)

N['CODE_STIF_RES'] = N['CODE_STIF_RES'].str.strip()
N = N[N["CODE_STIF_RES"] == "852"]                # Selecting rows from line N stations
N = N.drop(["CODE_STIF_TRNS", "ID_REFA_LDA", "lda"], axis = 1)
N = N.rename(columns = {"JOUR" : "date", "CODE_STIF_RES" : "network",  "CODE_STIF_ARRET" : "station_id", "LIBELLE_ARRET" : "station_name", "CATEGORIE_TITRE" : "pass_type", "NB_VALD" : "n_passengers"})
N['network'] = N['network'].map(network_dict)
N['n_passengers'] = N.apply(lambda row: 2 if row['n_passengers'] == "Moins de 5" else row['n_passengers'], axis = 1)         #Arbitrarily setting "< 5" to 2 validations
N['n_passengers'] = N['n_passengers'].astype("int")

N['year'] = N['date'].dt.year
N['day'] = N['date'].dt.day
N['month'] = N['date'].dt.month
N['day_name'] = N['date'].dt.day_name()
N['month_name'] = N['date'].dt.month_name()

N = N.pivot_table(index=['date', 'station_id', 'station_name', 'year', 'day', 'month', 'day_name', 'month_name'],
                                  columns='pass_type', values='n_passengers', aggfunc='sum').reset_index()
N.columns.name = None
N[["?", "AMETHYSTE", "AUTRE TITRE", "FGT", "IMAGINE R", "NAVIGO", "NAVIGO JOUR", "NON DEFINI", "TST"]] = N[["?", "AMETHYSTE", "AUTRE TITRE", "FGT", "IMAGINE R", "NAVIGO", "NAVIGO JOUR", "NON DEFINI", "TST"]].fillna(0).astype("int")

N = N.groupby('date').agg(aggregation).reset_index()
N['station_name'] = "line_N"
N[['station_id', "network"]] = "N/A"

N['social'] = N['FGT'] + N['TST']
N['other'] = N['?'] + N['AUTRE TITRE'] + N['NON DEFINI']
N = N.drop(['?', "AUTRE TITRE", "FGT", "NON DEFINI", "TST"], axis = 1)
N = N.rename(columns = {"AMETHYSTE" : "amethyste", "IMAGINE R" : "imagine_r", "NAVIGO": "navigo", "NAVIGO JOUR": "navigo_jour"})
N["daily_traffic"] = N["amethyste"] + N["imagine_r"] + N["navigo"] + N["navigo_jour"] + N ["social"] + N["other"]
N = N.drop('network', axis = 1)
S1 = ["January", "February", "March", "April", "May", "June"]
S2 = ["July", "August", "September", "October", "November", "December"]

N["semester"] = N.apply(lambda row: 1 if row['month_name'] in S1 else 2 if row['month_name'] in S2 else None, axis = 1)


N

,date,station_id,station_name,amethyste,imagine_r,navigo,navigo_jour,year,day,month,day_name,month_name,social,other,daily_traffic,semester
0,2015-01-01,N/A,line_N,354,3969,6281,0,2015,1,1,Thursday,January,1573,611,12788,1
1,2015-01-02,N/A,line_N,1015,9588,30174,0,2015,2,1,Friday,January,5148,1881,47806,1
2,2015-01-03,N/A,line_N,977,8656,18508,0,2015,3,1,Saturday,January,4322,1115,33578,1
3,2015-01-04,N/A,line_N,703,5471,12817,0,2015,4,1,Sunday,January,3090,1063,23144,1
4,2015-01-05,N/A,line_N,1295,24505,78767,0,2015,5,1,Monday,January,6797,4415,115779,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3098,2023-06-26,N/A,line_N,1628,24242,68656,383,2023,26,6,Monday,June,8773,4183,107865,1
3099,2023-06-27,N/A,line_N,1572,24608,71519,443,2023,27,6,Tuesday,June,8701,4065,110908,1
3100,2023-06-28,N/A,line_N,1569,25066,65647,418,2023,28,6,Wednesday,June,7649,3769,104118,1
3101,2023-06-29,N/A,line_N,1588,25494,71026,469,2023,29,6,Thursday,June,8846,4249,111672,1


<h4>Versailles</h4>

In [12]:
# 2015

df15_s1 = pd.read_csv('original_datasets/data-rf-2015/2015S1_NB_FER.csv', sep = ";")
df15_s1 = df15_s1[df15_s1['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

df15_s2 = pd.read_csv('original_datasets/data-rf-2015/2015S2_NB_FER.csv', sep = ";")
df15_s2 = df15_s2[df15_s2['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

# 2016

df16_s1 = pd.read_csv('original_datasets/data-rf-2016/2016S1_NB_FER.txt', sep = "\t")
df16_s1 = df16_s1[df16_s1['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

df16_s2 = pd.read_csv('original_datasets/data-rf-2016/2016S2_NB_FER.txt', sep = "\t")
df16_s2 = df16_s2[df16_s2['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

# 2017

df17_s1 = pd.read_csv('original_datasets/data-rf-2017/2017S1_NB_FER.txt', sep = "\t")
df17_s1 = df17_s1[df17_s1['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

df17_s2 = pd.read_csv('original_datasets/data-rf-2017/2017_S2_NB_FER.txt', sep = "\t")
df17_s2 = df17_s2[df17_s2['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

# 2018

df18_s1 = pd.read_csv('original_datasets/data-rf-2018/2018_S1_NB_FER.txt', sep = "\t")
df18_s1 = df18_s1[df18_s1['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

df18_s2 = pd.read_csv('original_datasets/data-rf-2018/2018_S2_NB_FER.txt', sep = "\t")
df18_s2 = df18_s2[df18_s2['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

# 2019

df19_s1 = pd.read_csv('original_datasets/data-rf-2019/2019_S1_NB_FER.txt', sep = "\t")
df19_s1 = df19_s1[df19_s1['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

df19_s2 = pd.read_csv('original_datasets/data-rf-2019/2019_S2_NB_FER.txt', sep = "\t")
df19_s2 = df19_s2[df19_s2['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

# 2020

df20_s1 = pd.read_csv('original_datasets/data-rf-2020/2020_S1_NB_FER.txt', sep = "\t")
df20_s1 = df20_s1[df20_s1['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

df20_s2 = pd.read_csv('original_datasets/data-rf-2020/2020_S2_NB_FER.txt', sep = "\t")
df20_s2 = df20_s2[df20_s2['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

# 2021

df21_s1 = pd.read_csv('original_datasets/data-rf-2021/2021_S1_NB_FER.txt', sep = "\t")
df21_s1 = df21_s1[df21_s1['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

df21_s2 = pd.read_csv('original_datasets/data-rf-2021/2021_S2_NB_FER.txt', sep = "\t")
df21_s2 = df21_s2[df21_s2['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

# 2022

df22_s1 = pd.read_csv('original_datasets/data-rf-2022/2022_S1_NB_FER.txt', sep = "\t")
df22_s1 = df22_s1[df22_s1['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

df22_s2 = pd.read_csv('original_datasets/data-rf-2022/2022_S2_NB_FER.txt', sep = ";")
df22_s2 = df22_s2[df22_s2['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

# 2023

df23_s1 = pd.read_csv('original_datasets/data-rf-2023/2023_S1_NB_FER.csv', sep = ";")
df23_s1 = df23_s1[df23_s1['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

df2 = pd.concat([df15_s1, df15_s2, df16_s1, df16_s2, df17_s1, df17_s2, df18_s1, df18_s2,
                df19_s1, df19_s2, df20_s1, df20_s2, df21_s1, df21_s2, df22_s1, df22_s2,
               ]).reset_index(drop = True)

df2['JOUR'] = pd.to_datetime(df2['JOUR'], format = '%d/%m/%Y')
df23_s1['JOUR'] = pd.to_datetime(df23_s1['JOUR'], format = '%Y-%m-%d')



In [13]:
daily = pd.concat([df2, df23_s1]).reset_index(drop = True)
daily = daily.drop(["CODE_STIF_TRNS", "CODE_STIF_RES", "ID_REFA_LDA", "lda"], axis = 1)
daily = daily.rename(columns = {"JOUR" : "date", "CODE_STIF_ARRET" : "station_id", "LIBELLE_ARRET" : "station_name", "CATEGORIE_TITRE" : "pass_type", "NB_VALD" : "n_passengers"})
daily['station_name'] = daily['station_id'].map(stif_code_reverse)
daily['n_passengers'] = daily.apply(lambda row: 2 if row['n_passengers'] == "Moins de 5" else row['n_passengers'], axis = 1)
daily['n_passengers'] = daily['n_passengers'].astype("int")
daily['year'] = daily['date'].dt.year
daily['day'] = daily['date'].dt.day
daily['month'] = daily['date'].dt.month
daily['day_name'] = daily['date'].dt.day_name()
daily['month_name'] = daily['date'].dt.month_name()
daily = daily.pivot_table(index=['date', 'station_id', 'station_name', 'year', 'day', 'month', 'day_name', 'month_name'],
                                  columns='pass_type', values='n_passengers', aggfunc='sum').reset_index()
daily.columns.name = None
daily[["?", "AMETHYSTE", "AUTRE TITRE", "FGT", "IMAGINE R", "NAVIGO", "NAVIGO JOUR", "NON DEFINI", "TST"]] = daily[["?", "AMETHYSTE", "AUTRE TITRE", "FGT", "IMAGINE R", "NAVIGO", "NAVIGO JOUR", "NON DEFINI", "TST"]].fillna(0).astype("int")
daily['social'] = daily['FGT'] + daily['TST']
daily['other'] = daily['?'] + daily['AUTRE TITRE'] + daily['NON DEFINI']
daily = daily.drop(['?', "AUTRE TITRE", "FGT", "NON DEFINI", "TST"], axis = 1)
daily = daily.rename(columns = {"AMETHYSTE" : "amethyste", "IMAGINE R" : "imagine_r", "NAVIGO": "navigo", "NAVIGO JOUR": "navigo_jour"})
daily["daily_traffic"] = daily["amethyste"] + daily["imagine_r"] + daily["navigo"] + daily["navigo_jour"] + daily["social"] + daily["other"]

S1 = ["January", "February", "March", "April", "May", "June"]
S2 = ["July", "August", "September", "October", "November", "December"]

daily["semester"] = daily.apply(lambda row: 1 if row['month_name'] in S1 else 2 if row['month_name'] in S2 else None, axis = 1)

daily = pd.concat([daily, idf, L, N]).reset_index(drop = True)
daily

,date,station_id,station_name,year,day,month,day_name,month_name,amethyste,imagine_r,navigo,navigo_jour,social,other,daily_traffic,semester
0,2015-01-01,566,Montreuil,2015,1,1,Thursday,January,16,77,111,0,22,9,235,1.0
1,2015-01-01,679,Porchefontaine,2015,1,1,Thursday,January,7,33,52,0,4,2,98,1.0
2,2015-01-01,884,Versailles Rive Droite,2015,1,1,Thursday,January,31,204,463,0,89,39,826,1.0
3,2015-01-01,885,Versailles Château Rive Gauche,2015,1,1,Thursday,January,28,133,398,0,67,21,647,1.0
4,2015-01-01,886,Versailles Chantiers,2015,1,1,Thursday,January,35,500,751,0,157,123,1566,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24223,2023-06-26,N/A,line_N,2023,26,6,Monday,June,1628,24242,68656,383,8773,4183,107865,1.0
24224,2023-06-27,N/A,line_N,2023,27,6,Tuesday,June,1572,24608,71519,443,8701,4065,110908,1.0
24225,2023-06-28,N/A,line_N,2023,28,6,Wednesday,June,1569,25066,65647,418,7649,3769,104118,1.0
24226,2023-06-29,N/A,line_N,2023,29,6,Thursday,June,1588,25494,71026,469,8846,4249,111672,1.0


In [29]:
gares = pd.read_csv('original_datasets/referentiel-gares-voyageurs.csv', sep = ";")
gares["Commune"] = gares["Commune"].dropna()
infos_v = gares[gares["Commune"] == 'Versailles'][["Longitude", 'Latitude', "Intitulé gare"]]
infos_v['stif_code'] = infos_v['Intitulé gare'].map(stif_code)
infos_v = infos_v.rename(columns = {"Longitude" : "longitude", "Latitude" : "latitude", "Intitulé gare" : "station_name", "stif_code" : "station_id"})
infos_v = infos_v.reset_index(drop = True)
infos_v = infos_v[['station_name', 'station_id', 'latitude', 'longitude']]

url_rive_droite = 'https://upload.wikimedia.org/wikipedia/commons/2/20/Versailles-Rive-Droite_Station.jpg'
url_chateau = 'https://upload.wikimedia.org/wikipedia/commons/e/e8/Gare_de_Versailles-Ch%C3%A2teau-Rive-Gauche_1.jpg'
url_montreuil = 'https://upload.wikimedia.org/wikipedia/commons/2/23/Versailles_Gare_de_Montreuil.JPG'
url_porchefontaine = 'https://upload.wikimedia.org/wikipedia/commons/thumb/c/c4/Versailles_Gare_de_Porchefontaine.JPG/1200px-Versailles_Gare_de_Porchefontaine.JPG'
url_chantiers = 'https://www.transilien.com/sites/transilien/files/styles/cards_crop/public/2020-08/Versailles%20Chantiers%20vignette.jpg?h=37336df8&itok=5B5lVowk'

address_chantiers = "4, rue de l'Abbé Rousseau 78000 Versailles"
address_chateau = 'Place Lyautey 78000 Versailles'
address_porchefontaine = 'Avenue de Porchefontaine 78000 Versailles'
address_montreuil = 'Rue de la Bonne Aventure 78000 Versailles'
address_rive_droite = "Rue du Maréchal Foch 78000 Versailles"

infos = {
    0 : [url_montreuil, address_montreuil],
    1 : [url_rive_droite, address_rive_droite],
    2 : [url_porchefontaine, address_porchefontaine],
    3 : [url_chateau, address_chateau],
    4 : [url_chantiers, address_chantiers]
}

for digit, info in infos.items():
    infos_v.loc[digit, 'station_img'] = info[0]
    infos_v.loc[digit, 'station_address'] = info[1]

In [15]:
daily.head(1)

,date,station_id,station_name,year,day,month,day_name,month_name,amethyste,imagine_r,navigo,navigo_jour,social,other,daily_traffic,semester
0,2015-01-01,566,Montreuil,2015,1,1,Thursday,January,16,77,111,0,22,9,235,1.0


In [16]:
idf.head(1)

,date,station_id,station_name,amethyste,imagine_r,navigo,navigo_jour,year,day,month,day_name,month_name,social,other,daily_traffic,semester
0,2015-01-01,N/A,all_idf,10286,56583,141278,0,2015,1,1,Thursday,January,56407,9718,274272,1


In [17]:
infos_v.head()

,station_name,station_id,latitude,longitude
0,Montreuil,566,48.806732,2.151169
1,Versailles Rive Droite,884,48.809652,2.134752
2,Porchefontaine,679,48.796938,2.154016
3,Versailles Château Rive Gauche,885,48.800364,2.128827
4,Versailles Chantiers,886,48.795826,2.135883


In [18]:
count_code = {
    "Versailles Rive Droite" : 87382861,
    "Versailles Château Rive Gauche" : 87393157,
    "Versailles Chantiers" : 87393009,
    "Porchefontaine" : 87393165,
    "Montreuil" : 87382879
}

In [19]:
count = pd.read_csv('original_datasets/comptage-transilien.csv', sep = ";")
count = count[count['Code Gare'].isin(count_code.values())]
count = count.rename(columns = {"Nom gare" : "station_name", "Code Gare" : "station_id", "Type jour" : "day_type", "Année" : "year", "Date de comptage" : "date", "Ligne" : "line", "Tranche horaire" : "time_period", "Montants" : "counts"})
count = count.drop(["Axe", "year", "date"], axis = 1)
count['station_name'] = count['station_id'].map({value:key for key,value in count_code.items()})
count['station_id'] = count['station_name'].map(stif_code)
count['day_type'] = count['day_type'].map({"JOB":'weekday', "Samedi": "saturday", "Dimanche":"sunday"})
count['time_period'] = count['time_period'].map({"Avant 6h" : "before 6am", "De 6h à 10h": "6am - 10am", "De 10h à 16h": "10am - 4pm", "De 16h à 20h":"4pm - 8pm" ,"Après 20h" : "after 8pm"})
count.head(50)

,station_name,station_id,day_type,line,time_period,counts
218,Porchefontaine,679,sunday,C,after 8pm,46
349,Porchefontaine,679,weekday,C,10am - 4pm,337
359,Porchefontaine,679,saturday,C,after 8pm,63
375,Versailles Chantiers,886,sunday,C,10am - 4pm,724
586,Porchefontaine,679,saturday,C,before 6am,7
652,Porchefontaine,679,saturday,C,4pm - 8pm,249
726,Versailles Chantiers,886,saturday,C,4pm - 8pm,1015
759,Porchefontaine,679,sunday,C,6am - 10am,62
773,Versailles Chantiers,886,saturday,C,10am - 4pm,1105
782,Versailles Chantiers,886,weekday,C,10am - 4pm,1343


In [34]:
traffic.to_csv('ef_v_traffic.csv', index = False)
daily.to_csv('ef_v_daily.csv', index = False)
infos_v.to_csv('ef_v_infos.csv', index = False)
count.to_csv('ef_v_count.csv', index = False)

In [21]:
traffic.head()

,station_name,station_id,origin_id,year,rank
0,Versailles Chantiers,886,531,2015,8
1,Versailles Château Rive Gauche,885,457,2015,41
2,Versailles Rive Droite,884,458,2015,83
3,Montreuil,566,423,2015,183
4,Porchefontaine,679,663,2015,264


In [22]:
daily.head()

,date,station_id,station_name,year,day,month,day_name,month_name,amethyste,imagine_r,navigo,navigo_jour,social,other,daily_traffic,semester
0,2015-01-01,566,Montreuil,2015,1,1,Thursday,January,16,77,111,0,22,9,235,1.0
1,2015-01-01,679,Porchefontaine,2015,1,1,Thursday,January,7,33,52,0,4,2,98,1.0
2,2015-01-01,884,Versailles Rive Droite,2015,1,1,Thursday,January,31,204,463,0,89,39,826,1.0
3,2015-01-01,885,Versailles Château Rive Gauche,2015,1,1,Thursday,January,28,133,398,0,67,21,647,1.0
4,2015-01-01,886,Versailles Chantiers,2015,1,1,Thursday,January,35,500,751,0,157,123,1566,1.0


In [30]:
infos_v.head()

,station_name,station_id,latitude,longitude
0,Montreuil,566,48.806732,2.151169
1,Versailles Rive Droite,884,48.809652,2.134752
2,Porchefontaine,679,48.796938,2.154016
3,Versailles Château Rive Gauche,885,48.800364,2.128827
4,Versailles Chantiers,886,48.795826,2.135883


In [24]:
count.head()

,station_name,station_id,day_type,line,time_period,counts
218,Porchefontaine,679,sunday,C,after 8pm,46
349,Porchefontaine,679,weekday,C,10am - 4pm,337
359,Porchefontaine,679,saturday,C,after 8pm,63
375,Versailles Chantiers,886,sunday,C,10am - 4pm,724
586,Porchefontaine,679,saturday,C,before 6am,7


,station_name,station_id,latitude,longitude,station_img,station_address
0,Montreuil,566,48.806732,2.151169,https://upload.wikimedia.org/wikipedia/commons...,Rue de la Bonne Aventure 78000 Versailles
1,Versailles Rive Droite,884,48.809652,2.134752,https://upload.wikimedia.org/wikipedia/commons...,Rue du Maréchal Foch 78000 Versailles
2,Porchefontaine,679,48.796938,2.154016,https://upload.wikimedia.org/wikipedia/commons...,Avenue de Porchefontaine 78000 Versailles
3,Versailles Château Rive Gauche,885,48.800364,2.128827,https://upload.wikimedia.org/wikipedia/commons...,Place Lyautey 78000 Versailles
4,Versailles Chantiers,886,48.795826,2.135883,https://www.transilien.com/sites/transilien/fi...,"4, rue de l'Abbé Rousseau 78000 Versailles"
